#  **Analisis Sentimen Ulasan Aplikasi MyXL dengan SVM di Google Play Store**

Review pada Google Play Store merupakan salah satu fitur yang digunakan untuk memberikan suatu penilaian terhadap suatu aplikasi. MyXL merupakan aplikasi self-service yang diberikan oleh PT XL Axiata Tbk pada Google Play Store yang berguna dalam proses memudahkan pengguna dalam melakukan kegiatan yang diberikan fitur-fitur seperti aktivasi paket internet, cek pulsa, cek sisa kuota, layanan FAQ dan Live Chat, dll. Namun review pada aplikasi MyXL tersebut hanya berupa teks tanpa arti tertentu dan terdapat beberapa pengguna yang memberikan rating yang tinggi namun ulasan yang diberikan merupakan review negatif, untuk itu diperlukan analisis yang dapat mengklasifikasikan review sebagai sentimen pengguna. 

Dalam penelitian ini dilakukan tahap scraping untuk pengumpulan data informasi ulasan pengguna aplikasi MyXL, setelah itu melakukan pelabelan terhadap data dan dikategorisasikan kedalam kelas positif, negatif dan netral. Selanjutnya text preprocessing untuk mengolah data dengan menyeleksi data dan mengubahnya menjadi data yang lebih terstruktur agar dapat digunakan sesuai kebutuhan penelitian. Setelah mendapatkan data hasil text preprocessing dilakukan pembobotan kata dengan memberikan nilai pada suatu kata dalam sebuah dokumen menggunakan metode Term Frequency – Inverse Document Frequency (TF-IDF). Kemudian dilakukan pengolahan dengan mengklasifikasi menggunakan algoritma Support Vector Machine (SVM).

**Manfaat dilakukan analisis sentimen adalah:**
1. Hasil analisis sentimen dapat dilihat oleh pengguna aplikasi MyXL dan agar dapat mengetahui opini pengguna lain.
2. Sebagai bahan pertimbangan PT XL Axiata Tbk untuk menyempurnakan dan mengimprovisasi aplikasi MyXL dengan hasil analisis yang didapatkan.

**1. Scraping Data**
 
Scraping data dilakukan dengan mengumpukan data ulasan pengguna aplikasi MyXL dari Google Play Store menggunakan bantuan dari library google-play-scraper dan IDE Google Collab. Data yang diambil berupa nama pemberi ulasan, nilai bintang yang diberikan, waktu ulasan dikirim dan isi review ulasan.

In [ ]:
!pip install google-play-scraper

In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews

In [ ]:
result, continuation_token = reviews(
    'com.apps.MyXL',
    lang='id', 
    country='id', 
    sort=Sort.MOST_RELEVANT, 
    count=1000, 
    filter_score_with=None
)
df_busu = pd.DataFrame(np.array(result),columns=['review'])
df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))

In [ ]:
new_df = df_busu[['userName', 'score','at', 'content']]
sorted_df = new_df.sort_values(by='at', ascending=False) 
sorted_df.to_csv("Ulasan My XL 1000 Data.csv", index = False)

**2. Pelabelan Dataset**

Pelabelan dataset dilakukan secara manual terhadap pelabelan data kedalam kelas kategori positif, negatif atau netral. Proses ini akan dilakukan dengan bantuan ahli bahasa Indonesia dan dilakukan pelabelan oleh setidaknya lebih dari dua orang.


**3. Case Folding**

Case folding dilakukan pengubahan seluruh huruf menjadi kecil (lowercase) yang ada pada dokumen. Tahap ini akan dibantu dengan bantuan library RegEx.

In [ ]:
import re
def cleaningulasan(ulasan):
  ulasan = re.sub(r'@[A-Za-a0-9]+',' ',ulasan)
  ulasan = re.sub(r'#[A-Za-z0-9]+',' ',ulasan)
  ulasan = re.sub(r"http\S+",' ',ulasan)
  ulasan = re.sub(r'[0-9]+',' ',ulasan)
  ulasan = re.sub(r"[-()\"#/@;:<>{}'+=~|.!?,_]", " ", ulasan)
  ulasan = ulasan.strip(' ')
  return ulasan
ulasan['Cleaning']= ulasan['Ulasan'].apply(cleaningulasan)

def clearEmoji(ulasan):
    return ulasan.encode('ascii', 'ignore').decode('ascii')
ulasan['HapusEmoji']= ulasan['Cleaning'].apply(clearEmoji)

def replaceTOM(ulasan):
    pola = re.compile(r'(.)\1{2,}', re.DOTALL)
    return pola.sub(r'\1', ulasan)
ulasan['3/Lebih']= ulasan['HapusEmoji'].apply(replaceTOM)

def casefoldingText(ulasan):
  ulasan = ulasan.lower()
  return ulasan
ulasan['CaseFolding']= ulasan['3/Lebih'].apply(casefoldingText)

**4. Tokenizing**

Hasil proses case folding, kalimat akan diproses dengan menguraikannya menjadi token-token atau kata-kata. Pada tahap ini akan dibantu dengan library NLTK.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
def tokenizingText(ulasan):
  ulasan = word_tokenize(ulasan)
  return ulasan
ulasan['Tokenizing']= ulasan['CaseFolding'].apply(tokenizingText)

**5. Formalisasi**

Tahap formalisasi dilakukan untuk pengubah penggunaan kata tidak baku menjadi baku sesuai dengan KBBI. Proses akan menggunakan file dataset slangwords yang berisi kata slang yang nanti akan diubah menjadi baku. Tahap ini dibantu dengan library RegEx. 

In [ ]:
def convertToSlangword(ulasan):
    kamusSlang = eval(open("slangwords.txt").read())
    pattern = re.compile(r'\b( ' + '|'.join (kamusSlang.keys())+r')\b')
    content = []
    for kata in ulasan:
        filterSlang = pattern.sub(lambda x: kamusSlang[x.group()],kata)
        content.append(filterSlang.lower())
    ulasan = content
    return ulasan

ulasan['Formalisasi'] = ulasan['Tokenizing'].apply(convertToSlangword)

**6. Stopword Removal**

Hasil proses formalisasi kemudian akan dilakukan seleksi kata yang tidak penting dan menghapus kata tersebut. Tahap ini dibantu dengan library NLTK.

In [ ]:
from nltk.corpus import stopwords

daftar_stopword = stopwords.words('indonesian')
# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
daftar_stopword.extend(["yg","dg","rt"])
daftar_stopword = set(daftar_stopword)

def stopwordText(words):
 return [word for word in words if word not in daftar_stopword]

ulasan['Stopword Removal'] = ulasan['Formalisasi'].apply(stopwordText)

**7. Stemming**

Proses stemming dilakukan perubahan kata yang berimbuhan menjadi kata dasar. Tahap ini dibantu dengan library Sastrawi dan Swifter.

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in ulasan['Stopword Removal']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
def stemmingText(document):
    return [term_dict[term] for term in document]

ulasan['Stemming'] = ulasan['Stopword Removal'].swifter.apply(stemmingText)

**8. Pembobotan Kata dengan TF-IDF**

Pada tahap ini dilakukan pembobotan kata dari hasil stemming dengan metode Term Inverse Document Frequency (TF-IDF).Metode TF-IDF digunakan untuk mengetahui seberapa sering suatu kata muncul di dalam dokumen. Tahap ini dibantu dengan library sklearn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = df['Stemming']
Y = df['Sentimen']

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2)

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

**9. Klasifikasi dengan SVM**

Klasifikasi ulasan pengguna dilakukan menggunakan algoritma Support Vector Machine yang akan dibantu dengan library Scikit-Learn. Proses klasifikasi menggunakan nilai data latih dan data uji sebesar 80%:20%, dilakukan percobaan sebanyak 5x dan kernel akan menggunakan kernel rbf dan kernel linear.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

SVM = svm.SVC(kernel='rbf')    #Jika dengan Kernel RBF
SVM = svm.SVC(kernel='linear') #Jika dengan Kernel Linear
SVM.fit(x_train,y_train)

acc_score = cross_val_score(SVM, x_train, y_train, cv=5, scoring='accuracy')
pre_score = cross_val_score(SVM, x_train, y_train, cv=5, scoring='precision_macro')
rec_score = cross_val_score(SVM, x_train, y_train, cv=5, scoring='recall_macro')
f_score = cross_val_score(SVM, x_train, y_train, cv=5, scoring='f1_macro')

print('Hasil Accuracy : %s' % (acc_score))
print('Hasil Rata - Rata Accuracy : %s' % acc_score.mean())
print('Hasil Precision : %s' % (pre_score))
print('Hasil Rata - Rata Precision : %s' % pre_score.mean())
print('Hasil Recall : %s' % (rec_score))
print('Hasil Rata - Rata Recall : %s' % rec_score.mean())
print('Hasil F-Measure : %s' % (f_score))
print('Hasil Rata - Rata F-Measure : %s' % f_score.mean())

# Hasil Klasifikasi

Hasil klasifikasi pertama didapatkan bahwa pada pengujian pertama penggunaan data utuh berjumlah 1000 data dengan penggunaan data seimbang 480 data pada sentimen 3 kelas didapatkan bahwa hasil akurasi terbaik pada data utuh dengan pengujian kernel Linear menghasilkan nilai akurasi sebesar 71%.Hal ini dapat dikatan bahwa pada hasil pengujian ini semakin banyak data yang digunakan maka hasil yang didapatkan akan semakin tinggi, tanpa memedulikan data tersebut tidak seimbang atau seimbang.



Hasil klasifikasi kedua akan diuji nilai kernel terbaik dengan penggunaan sentimen 3 kelas terbaik sebelumnya dengan penggunaan sentimen 2 kelas, mendapatkan hasil bahwa dari pengujian yang dilakukan penggunaan nilai kernel pada kedua skenario hasil tertinggi menggunakan kernel linear yang sebesar 89% pada sentimen 2 kelas. Dapat dikatakan bahwa nilai kernel linear paling cocok untuk digunakan dalam pengujian ini karena bisa mendapatkan hasil lebih baik dari penggunaan kernel rbf.



Ketiga dilakukan pengujian evaluasi performa pada hasil klasifikasi sebelumnya, didapatkan utnuk evaluasi performa tersebut hasil terbaik dari pengujian pada sentimen 2 kelas didapatkan nilai rata-rata accuracy sebesar 89%, sedangkan pada pengujian sentimen 3 kelas hanya sebesar 71%. Nilai rata-rata presicion terbaik pada pengujian sentimen 2 kelas sebesar 90%, sedangkan pada pengujian sentimen 3 kelas hanya sebesar 70%. Nilai rata-rata recall terbaik pada pengujian sentimen 2 kelas sebesar 76%, sedangkan pada pengujian sentimen 3 kelas hanya sebesar 55%. Nilai rata-rata f-measure terbaik pada pengujian sentimen 2 kelas sebesar 80%, sedangkan pada pengujian sentimen 3 kelas hanya sebesar 58%. Penggunaan sentimen 2 kelas mendapatkan hasil akurasi lebih baik dibandingkan dengan sentimen 3 kelas. Hal ini dikarenakan proses klasifikasi pada sentimen 3 kelas yang kompleks karena terdapat 3 kategori kelas yaitu negatif, netral dan positif, sedangkan pada sentimen 2 kelas tidak membutuhkan proses  klasifikasi yang kompleks karena hanya terdapat 2 kategori kelas yaitu negatif dan positif saja.



Dapat disimpulkan bahwa penggunaan data yang lebih banyak dapat membantu untuk mendapatkan hasil klasifikasi lebih maksimal, penggunaan kernel yang digunakan akan sangat berpengaruh terhadap hasil akurasi maka dari perlu untuk mengguji dengan berbagai kernel agar dapat mengetahui kernel terbaik, dan penggunaan kelas jumlah kelas akan mempengaruhi proses klasifikasi semakin banyak kelas yang digunakan maka proses akan semakin komplek dan hasil akurasi menurun.

Untuk hasil klasifikasi ini dapat dilihat melalui visualisasi menggunakan Google Data Studio berikut:

https://datastudio.google.com/u/0/reporting/1fc0efc9-fa9e-4554-8b82-b959cf5e9815/page/4A5wC

# **Akhir Artikel**

Semoga dari penulisan artikel ini dapat membantu kalian yang membutuhkan referensi terkait hal analisis sentimen. Kurang lebihnya mohon maaf dan terima kasih!